In [1]:
from bs4 import BeautifulSoup
import requests, yaml, pytz
import datetime as dt
import pandas as pd

In [2]:
efc_url = "https://www.efinancialcareers.co.uk/jobs/in-United-Kingdom"

url = efc_url

hr_posted = 0.5
min_posted = hr_posted * 60

time_now = dt.datetime.now().astimezone(pytz.utc)

In [ ]:
def api_url(page):
    with open("vars.yaml", "r") as f:
        vars = yaml.load(f, Loader = yaml.FullLoader)

    url, iter = "", 0
    for key, val in vars.items():
        if iter == 0: pass
        elif iter == 4: url += val + str(page) + "&"
        else: url += val

        iter += 1
    return url

In [3]:
def parser():
    headers = headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
    content = requests.get(url, headers).content
    soup = BeautifulSoup(content, "html.parser")

    return soup

In [4]:
def job_card():
    soup = parser()
    
    efc_jobcard = soup.find('efc-job-search-results').find_all('div', {'class': 'card-info'})
        
    return efc_jobcard

In [5]:
def first_page():
    jobcard = job_card()

    job_title, company, job_link, location, position_type, work_arrangement, salary, posted_on = [], [], [], [], [], [], [], []
    job_dct = {}

    pg = 1
    diff_mins = 0
    for item in jobcard:
        if diff_mins > min_posted: break

        job = item.find('h3').text

        job_title.append(job)
        company.append(item.find('div', {'class': 'font-body-3'}).text.lstrip().rstrip())
        job_link.append(item.find('a').get('href'))
        location.append(item.find('span', {'class': 'dot-divider'}).text)
        position_type.append(item.find('span', {'class': 'dot-divider ng-star-inserted'}).text)

        try: work_arrangement.append(item.find('span', {'class': 'dot-divider-after ng-star-inserted'}).text)
        except: work_arrangement.append(None)

        try: salary.append(item.find('span', {'class': 'dot-divider-after last-job-criteria ng-star-inserted'}).text)
        except: salary.append(None)

        pdt = posted_date(job, pg)
        posted_on.append(pdt)

    job_dct = {"Position": job_title, "Company": company, "Link": job_link, "Location": location, "Position Type": position_type,
               "Work Arrangement": work_arrangement, "Salary": salary, "Posted Date": posted_on}
    final_df = pd.DataFrame(job_dct)
    
    return final_df

In [ ]:
from dateutil import parser

def posted_date(job, page):
    pg_url = api_url(page)
    data = requests.get(pg_url).json()['data']

    for i in range(0, len(data)):
        if job == data[i]['title']:
            posted_date = parser.parse(data[i]['postedDate'])
            break
    
    return posted_date


In [6]:
test = first_page()
print(test)



                                             Position              Company  \
0                 Brand, Campaign and Content Manager  Canada Life Limited   
1                                            Engineer  Canada Life Limited   
2                              Assistant Fund Manager  Canada Life Limited   
3                     Operational Procurement Analyst  Canada Life Limited   
4   Cloud Solution Engineer/ Software engineer wit...      McCabe & Barton   
5           Head of Procurement & Supplier Governance  Canada Life Limited   
6               Head of TPRM & Operational Resilience  Canada Life Limited   
7             Technology Risk Senior Business Partner  Canada Life Limited   
8      Operational Resilience Senior Business Partner  Canada Life Limited   
9                         Business Resilience Analyst  Canada Life Limited   
10    Senior Operational Risk Analyst- Energy Trading       Charles Levick   
11  High Performance C++ Developer - London- Prop ...        Oxf